![Modelo](../data/img/the-queens-gambit-netflix%20copia.jpg)


In [31]:
import pandas as pd
import re


In [32]:
df = pd.read_csv("../data/raw/chess_games.csv")

df.head()

,Event,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
0,Classical,eisaaaa,HAMID449,1-0,2016.06.30,22:00:01,1901,1896,11.0,-11.0,D10,Slav Defense,300+5,Time forfeit,1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1,Blitz,go4jas,Sergei1973,0-1,2016.06.30,22:00:01,1641,1627,-11.0,12.0,C20,King's Pawn Opening: 2.b3,300+0,Normal,1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2,Blitz tournament,Evangelistaizac,kafune,1-0,2016.06.30,22:00:02,1647,1688,13.0,-13.0,B01,Scandinavian Defense: Mieses-Kotroc Variation,180+0,Time forfeit,1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....
3,Correspondence,Jvayne,Wsjvayne,1-0,2016.06.30,22:00:02,1706,1317,27.0,-25.0,A00,Van't Kruijs Opening,-,Normal,1. e3 Nf6 2. Bc4 d6 3. e4 e6 4. Nf3 Nxe4 5. Nd...
4,Blitz tournament,kyoday,BrettDale,0-1,2016.06.30,22:00:02,1945,1900,-14.0,13.0,B90,"Sicilian Defense: Najdorf, Lipnitsky Attack",180+0,Time forfeit,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...


In [33]:
df['AN']

0          1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1          1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2          1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....
3          1. e3 Nf6 2. Bc4 d6 3. e4 e6 4. Nf3 Nxe4 5. Nd...
4          1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...
                                 ...                        
6256179    1. e4 c5 2. Nf3 d6 3. d4 Qa5+ 4. Bd2 Qc7 5. Bc...
6256180    1. e4 e5 2. Nf3 h6 3. Nc3 d6 4. Bc4 Qf6 5. Nd5...
6256181    1. e4 { [%eval 0.22] } 1... c5 { [%eval 0.35] ...
6256182    1. Nf3 d5 2. c4 dxc4 3. Qa4+ c6 4. Qxc4 Nf6 5....
6256183    1. e4 e6 2. d4 d6 3. Nc3 c6 4. Nf3 Be7 5. Bc4 ...
Name: AN, Length: 6256184, dtype: object

In [34]:
df['AN'].str.contains(r'\[%eval', na=False).value_counts()

# Nos quedaremos solo con las partidas con evaluaciones para entrenar al modelo

AN
False    5455688
True      800496
Name: count, dtype: int64

In [35]:
df_eval = pd.read_csv("../data/raw/partidas_SF.csv")

df_eval.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800496 entries, 0 to 800495
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Event            800496 non-null  object 
 1   White            800496 non-null  object 
 2   Black            800496 non-null  object 
 3   Result           800496 non-null  object 
 4   UTCDate          800496 non-null  object 
 5   UTCTime          800496 non-null  object 
 6   WhiteElo         800496 non-null  int64  
 7   BlackElo         800496 non-null  int64  
 8   WhiteRatingDiff  799339 non-null  float64
 9   BlackRatingDiff  799339 non-null  float64
 10  ECO              800496 non-null  object 
 11  Opening          800496 non-null  object 
 12  TimeControl      800496 non-null  object 
 13  Termination      800496 non-null  object 
 14  AN               800496 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 91.6+ MB


In [36]:
df_eval['Event'].value_counts()

Event
 Classical                224518
 Blitz                    218213
 Bullet                   185885
 Bullet tournament         93777
 Blitz tournament          48977
 Classical tournament      25706
 Correspondence             3412
Classical                      3
Bullet                         2
Blitz                          1
Classical tournament           1
Bullet tournament              1
Name: count, dtype: int64

In [37]:
# Como la columna Event tiene valores mal equilibrados entre partidas de Torneo y amistosas la borro del
# DF porque no me parece que pueda ser representativa, lo mismo con UTCDate y UTCTime.
# Y como hay muchisimas partidas eliminamos los nulos de todas la columnas, y borro lo valores nulos

df_chess= df_eval.drop(['Event','UTCDate','UTCTime'], axis=1).dropna()

df_chess.info()

<class 'pandas.core.frame.DataFrame'>
Index: 799339 entries, 0 to 800495
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   White            799339 non-null  object 
 1   Black            799339 non-null  object 
 2   Result           799339 non-null  object 
 3   WhiteElo         799339 non-null  int64  
 4   BlackElo         799339 non-null  int64  
 5   WhiteRatingDiff  799339 non-null  float64
 6   BlackRatingDiff  799339 non-null  float64
 7   ECO              799339 non-null  object 
 8   Opening          799339 non-null  object 
 9   TimeControl      799339 non-null  object 
 10  Termination      799339 non-null  object 
 11  AN               799339 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 79.3+ MB


In [38]:
df_chess.head()

,White,Black,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,AN
0,chm00002,jan666,1-0,1761,1754,12.0,-12.0,C30,"King's Gambit Declined, Queen's Knight Defense",180+0,Time forfeit,1. e4 { [%eval 0.27] } 1... e5 { [%eval 0.27] ...
1,Damianlamy,BaderBombs,1-0,2134,2128,12.0,-11.0,B21,"Sicilian Defense: Smith-Morra Gambit Accepted,...",300+0,Normal,1. e4 { [%eval 0.31] } 1... c5 { [%eval 0.37] ...
2,jjj333,aldoelapache,1-0,1964,1970,11.0,-12.0,B00,Owen Defense,60+0,Time forfeit,1. e4 { [%eval 0.27] } 1... b6 { [%eval 0.51] ...
3,otger,gmvladimir,0-1,1515,1540,-11.0,10.0,C42,Petrov's Defense,300+0,Normal,1. e4 { [%eval 0.22] } 1... e5 { [%eval 0.25] ...
4,b0nker2,erdema,1-0,1422,1403,11.0,-10.0,D00,Queen's Pawn Game: Mason Attack,300+0,Normal,1. d4 { [%eval 0.24] } 1... d5 { [%eval 0.14] ...


In [39]:
print(df_chess['White'].value_counts())

# En principio estas columnas para un modelo predictivo general no nos aportaría nada, con lo que las 
# eliminaré del df principal.
# Podría dejarlas para otro modelo que te dijera estadisticas de determinado jugador, tipo de juego, ELO,
# y otras stats para por ejemplo realizar una predicción contra él.

White
pavelpavel              867
Evenflow                799
zafarynl                583
potsy722                573
palang1359              558
                       ... 
Zwickzwackzwieback        1
lakefront75               1
silvificknigthchess9      1
karka                     1
KarjalanSotilas           1
Name: count, Length: 85400, dtype: int64


In [40]:
cuenta_blancas = df_chess['White'].value_counts()
jugadores_frecuentes_blancas = cuenta_blancas[cuenta_blancas > 50].index

cuenta_negras = df_chess['Black'].value_counts()
jugadores_frecuentes_negras = cuenta_negras[cuenta_negras > 50].index

df_chess = df_chess[
    (df_chess['White'].isin(jugadores_frecuentes_blancas)) &
    (df_chess['Black'].isin(jugadores_frecuentes_negras))
]

In [41]:
df_chess.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80523 entries, 7 to 800494
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   White            80523 non-null  object 
 1   Black            80523 non-null  object 
 2   Result           80523 non-null  object 
 3   WhiteElo         80523 non-null  int64  
 4   BlackElo         80523 non-null  int64  
 5   WhiteRatingDiff  80523 non-null  float64
 6   BlackRatingDiff  80523 non-null  float64
 7   ECO              80523 non-null  object 
 8   Opening          80523 non-null  object 
 9   TimeControl      80523 non-null  object 
 10  Termination      80523 non-null  object 
 11  AN               80523 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 8.0+ MB


In [42]:
print(df_chess['Result'].value_counts())
# Está bastante equilibrado entre victorias blancas y negras

Result
1-0        39990
0-1        39556
1/2-1/2      977
Name: count, dtype: int64


In [43]:
# Comprobamos el número de partidas registradas para cada apertura
df_chess['ECO'].value_counts()

ECO
A00    8455
A40    5440
B01    4151
C00    3992
B00    2931
       ... 
E05       1
E88       1
A89       1
A74       1
E19       1
Name: count, Length: 418, dtype: int64

In [44]:
# Como hay mucha fragmentación, y como sé que la letra (A, B, C, D...) representa a un tipo concreto
# de apertura creo una columna nueva solo con el valor del tipo

df_chess['ECO_Family'] = df_chess['ECO'].str[0]
print(df_chess['ECO_Family'].value_counts())
df_chess.drop(['ECO'],axis=1)

ECO_Family
A    29315
B    23335
C    17304
D     9258
E     1311
Name: count, dtype: int64


,White,Black,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,Opening,TimeControl,Termination,AN,ECO_Family
7,edushi,tiltCHIMP,0-1,1520,1791,-5.0,4.0,Sicilian Defense: Closed,60+0,Normal,1. e4 { [%eval 0.31] } 1... c5 { [%eval 0.39] ...,B
47,Spyandy,Pablo_M,1-0,1801,1628,7.0,-6.0,English Opening: Anglo-Gruenfeld Defense,60+0,Normal,1. c4 { [%eval 0.08] } 1... Nf6 { [%eval 0.1] ...,A
135,lichinut,borgogniels,0-1,1836,1943,-8.0,7.0,Hungarian Opening,30+0,Time forfeit,1. g3 { [%eval -0.16] } 1... e5 { [%eval -0.08...,A
138,Julinno38,maestro_aberbach,1-0,1928,1738,6.0,-5.0,"English Opening: King's English Variation, Two...",180+0,Normal,1. c4 { [%eval 0.06] } 1... e5 { [%eval 0.07] ...,A
141,MINSK_VAMPIRE,fsanna,0-1,1873,1873,-10.0,10.0,Van Geet Opening,0+1,Time forfeit,1. Nc3 { [%eval -0.07] } 1... e6 { [%eval 0.24...,A
...,...,...,...,...,...,...,...,...,...,...,...,...
800375,max_power,Clayderman,0-1,1821,2026,-6.0,6.0,"Sicilian Defense: Modern Variations, Tartakower",300+0,Normal,1. e4 { [%eval 0.31] } 1... c5 { [%eval 0.26] ...,B
800407,HardWoodPusher,ogor32,1-0,1992,1773,5.0,-5.0,Sicilian Defense: Closed Variation,0+1,Time forfeit,1. e4 { [%eval 0.19] } 1... c5 { [%eval 0.29] ...,B
800419,The_Ingenious,dynamicishwar,0-1,1769,1627,-17.0,18.0,Philidor Defense,60+0,Time forfeit,1. e4 { [%eval 0.19] } 1... d6 { [%eval 0.48] ...,C
800447,NimzoBros,amichai,1-0,1918,1787,8.0,-7.0,Modern Defense: Norwegian Defense,30+0,Time forfeit,1. d4 { [%eval 0.26] } 1... Nf6 { [%eval 0.16]...,B


In [45]:
# Fonction pour extraire les évaluations des coups et les convertir en une liste.

def extraer_evals(un_texte):
    evals = []
    start = 0
    while True:
        st = un_texte.find('[%eval ', start)
        if st == -1:
            break
        end_idx = un_texte.find(']', st)
        if end_idx == -1:
            break
        
        eval_str = un_texte[st + len('[%eval '): end_idx].strip()
        
        if eval_str.startswith('#'):
            if eval_str.startswith('#-'):
                evals.append(-100.0)
            else:
                evals.append(100.0)
        else:
            try:
                evals.append(float(eval_str))
            except ValueError:
                evals.append(None)
        
        start = end_idx + 1
    return evals

# Essaie avec une ligne qui contient des évaluations.
fila_test = df_chess[df_chess['AN'].str.contains('%eval')].iloc[2]  # Prends la première ligne avec des évaluations
print(fila_test['AN'][:500])  # Nous affichons les 500 premiers caractères pour vérifier
print(extraer_evals(fila_test['AN']))


1. g3 { [%eval -0.16] } 1... e5 { [%eval -0.08] } 2. Bg2 { [%eval -0.31] } 2... Nc6 { [%eval -0.13] } 3. d3 { [%eval -0.32] } 3... d5 { [%eval -0.27] } 4. e4 { [%eval -0.46] } 4... dxe4 { [%eval -0.44] } 5. dxe4?! { [%eval -1.27] } 5... Nf6? { [%eval 0.0] } 6. Qxd8+ { [%eval -0.09] } 6... Kxd8 { [%eval 0.0] } 7. Nc3?! { [%eval -0.56] } 7... Be6?! { [%eval 0.0] } 8. Bg5 { [%eval 0.0] } 8... Bb4 { [%eval 0.43] } 9. O-O-O+ { [%eval 0.42] } 9... Ke7?! { [%eval 1.19] } 10. Nd5+ { [%eval 1.18] } 10...
[-0.16, -0.08, -0.31, -0.13, -0.32, -0.27, -0.46, -0.44, -1.27, 0.0, -0.09, 0.0, -0.56, 0.0, 0.0, 0.43, 0.42, 1.19, 1.18, 1.18, 1.4, 3.41, 3.42, 3.73, 3.73, 8.45, 5.32, 5.47, 5.25, 8.37, 7.87, 12.36, 12.17, 13.0, 8.53, 13.0, 12.76, 14.81, 13.68, 19.39, 19.04, 19.11, 18.7, 100.0, 100.0, 100.0, 55.55, 55.51, 54.71, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 56.04, 50.74]


In [46]:
# 1. Filtra partidas que tienen evaluaciones
df_evals = df_chess[df_chess['AN'].str.contains('%eval', na=False)].copy()

# 2. Extrae todas las evaluaciones de cada partida
df_evals['evals'] = df_evals['AN'].apply(extraer_evals)

# 3. Calcula apertura, medio juego y final
from statistics import mean

def evals_por_fases(evals):
    n = len(evals)
    if n == 0:
        return None, None, None
    tercio = n // 3 or 1
    apertura = mean([e for e in evals[:tercio] if e is not None]) if evals[:tercio] else None
    medio = mean([e for e in evals[tercio:2*tercio] if e is not None]) if evals[tercio:2*tercio] else None
    final = mean([e for e in evals[2*tercio:] if e is not None]) if evals[2*tercio:] else None
    return apertura, medio, final

df_evals[['eval_apertura', 'eval_medio', 'eval_final']] = df_evals['evals'].apply(
    lambda x: pd.Series(evals_por_fases(x))
)


In [47]:
df_evals= df_evals.drop(['WhiteRatingDiff','BlackRatingDiff', 'Opening'], axis=1)

In [48]:
df_evals.head()

,White,Black,Result,WhiteElo,BlackElo,ECO,TimeControl,Termination,AN,ECO_Family,evals,eval_apertura,eval_medio,eval_final
7,edushi,tiltCHIMP,0-1,1520,1791,B23,60+0,Normal,1. e4 { [%eval 0.31] } 1... c5 { [%eval 0.39] ...,B,"[0.31, 0.39, 0.36, 0.28, -0.16, -0.1, -0.17, -...",-0.080000,-1.833077,-12.603571
47,Spyandy,Pablo_M,1-0,1801,1628,A16,60+0,Normal,1. c4 { [%eval 0.08] } 1... Nf6 { [%eval 0.1] ...,A,"[0.08, 0.1, 0.04, 0.24, 0.24, 0.27, 0.26, 0.21...",0.132308,0.166923,3.389333
135,lichinut,borgogniels,0-1,1836,1943,A00,30+0,Time forfeit,1. g3 { [%eval -0.16] } 1... e5 { [%eval -0.08...,A,"[-0.16, -0.08, -0.31, -0.13, -0.32, -0.27, -0....",0.015500,8.806000,78.609091
138,Julinno38,maestro_aberbach,1-0,1928,1738,A22,180+0,Normal,1. c4 { [%eval 0.06] } 1... e5 { [%eval 0.07] ...,A,"[0.06, 0.07, 0.12, 0.09, -0.11, 0.0, -0.21, 0....",0.132381,1.323333,34.990952
141,MINSK_VAMPIRE,fsanna,0-1,1873,1873,A00,0+1,Time forfeit,1. Nc3 { [%eval -0.07] } 1... e6 { [%eval 0.24...,A,"[-0.07, 0.24, -0.55, -0.22, -0.54, 0.0, -0.22,...",-0.053158,-2.251579,3.608500


In [19]:
df_evals.to_csv("../data/raw/partidas_final.csv", index=False)

In [20]:
df_1 = pd.read_csv('../data/raw/partidas_final.csv')
df_1.head()

,White,Black,Result,WhiteElo,BlackElo,ECO,TimeControl,Termination,AN,ECO_Family,evals,eval_apertura,eval_medio,eval_final
0,edushi,tiltCHIMP,0-1,1520,1791,B23,60+0,Normal,1. e4 { [%eval 0.31] } 1... c5 { [%eval 0.39] ...,B,"[0.31, 0.39, 0.36, 0.28, -0.16, -0.1, -0.17, -...",-0.080000,-1.833077,-12.603571
1,Spyandy,Pablo_M,1-0,1801,1628,A16,60+0,Normal,1. c4 { [%eval 0.08] } 1... Nf6 { [%eval 0.1] ...,A,"[0.08, 0.1, 0.04, 0.24, 0.24, 0.27, 0.26, 0.21...",0.132308,0.166923,3.389333
2,lichinut,borgogniels,0-1,1836,1943,A00,30+0,Time forfeit,1. g3 { [%eval -0.16] } 1... e5 { [%eval -0.08...,A,"[-0.16, -0.08, -0.31, -0.13, -0.32, -0.27, -0....",0.015500,8.806000,78.609091
3,Julinno38,maestro_aberbach,1-0,1928,1738,A22,180+0,Normal,1. c4 { [%eval 0.06] } 1... e5 { [%eval 0.07] ...,A,"[0.06, 0.07, 0.12, 0.09, -0.11, 0.0, -0.21, 0....",0.132381,1.323333,34.990952
4,MINSK_VAMPIRE,fsanna,0-1,1873,1873,A00,0+1,Time forfeit,1. Nc3 { [%eval -0.07] } 1... e6 { [%eval 0.24...,A,"[-0.07, 0.24, -0.55, -0.22, -0.54, 0.0, -0.22,...",-0.053158,-2.251579,3.608500


In [21]:
df_1['Result'] = df_1['Result'].map({
    '1-0': 1,
    '1/2-1/2': 0,
    '0-1': -1})


In [22]:
df_1.head(2)

,White,Black,Result,WhiteElo,BlackElo,ECO,TimeControl,Termination,AN,ECO_Family,evals,eval_apertura,eval_medio,eval_final
0,edushi,tiltCHIMP,-1,1520,1791,B23,60+0,Normal,1. e4 { [%eval 0.31] } 1... c5 { [%eval 0.39] ...,B,"[0.31, 0.39, 0.36, 0.28, -0.16, -0.1, -0.17, -...",-0.080000,-1.833077,-12.603571
1,Spyandy,Pablo_M,1,1801,1628,A16,60+0,Normal,1. c4 { [%eval 0.08] } 1... Nf6 { [%eval 0.1] ...,A,"[0.08, 0.1, 0.04, 0.24, 0.24, 0.27, 0.26, 0.21...",0.132308,0.166923,3.389333


In [23]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80523 entries, 0 to 80522
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   White          80523 non-null  object 
 1   Black          80523 non-null  object 
 2   Result         80523 non-null  int64  
 3   WhiteElo       80523 non-null  int64  
 4   BlackElo       80523 non-null  int64  
 5   ECO            80523 non-null  object 
 6   TimeControl    80523 non-null  object 
 7   Termination    80523 non-null  object 
 8   AN             80523 non-null  object 
 9   ECO_Family     80523 non-null  object 
 10  evals          80523 non-null  object 
 11  eval_apertura  80523 non-null  float64
 12  eval_medio     80523 non-null  float64
 13  eval_final     80523 non-null  float64
dtypes: float64(3), int64(3), object(8)
memory usage: 8.6+ MB


In [24]:
def limpiar_anotaciones(an_str):
    return re.sub(r'[!?]+', '', an_str)

df_1['AN'] = df_1['AN'].apply(limpiar_anotaciones)


In [25]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80523 entries, 0 to 80522
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   White          80523 non-null  object 
 1   Black          80523 non-null  object 
 2   Result         80523 non-null  int64  
 3   WhiteElo       80523 non-null  int64  
 4   BlackElo       80523 non-null  int64  
 5   ECO            80523 non-null  object 
 6   TimeControl    80523 non-null  object 
 7   Termination    80523 non-null  object 
 8   AN             80523 non-null  object 
 9   ECO_Family     80523 non-null  object 
 10  evals          80523 non-null  object 
 11  eval_apertura  80523 non-null  float64
 12  eval_medio     80523 non-null  float64
 13  eval_final     80523 non-null  float64
dtypes: float64(3), int64(3), object(8)
memory usage: 8.6+ MB


In [26]:
chess = df_1.drop(['eval_apertura', 'eval_medio', 'eval_final'], axis=1)

In [27]:
chess.head()

,White,Black,Result,WhiteElo,BlackElo,ECO,TimeControl,Termination,AN,ECO_Family,evals
0,edushi,tiltCHIMP,-1,1520,1791,B23,60+0,Normal,1. e4 { [%eval 0.31] } 1... c5 { [%eval 0.39] ...,B,"[0.31, 0.39, 0.36, 0.28, -0.16, -0.1, -0.17, -..."
1,Spyandy,Pablo_M,1,1801,1628,A16,60+0,Normal,1. c4 { [%eval 0.08] } 1... Nf6 { [%eval 0.1] ...,A,"[0.08, 0.1, 0.04, 0.24, 0.24, 0.27, 0.26, 0.21..."
2,lichinut,borgogniels,-1,1836,1943,A00,30+0,Time forfeit,1. g3 { [%eval -0.16] } 1... e5 { [%eval -0.08...,A,"[-0.16, -0.08, -0.31, -0.13, -0.32, -0.27, -0...."
3,Julinno38,maestro_aberbach,1,1928,1738,A22,180+0,Normal,1. c4 { [%eval 0.06] } 1... e5 { [%eval 0.07] ...,A,"[0.06, 0.07, 0.12, 0.09, -0.11, 0.0, -0.21, 0...."
4,MINSK_VAMPIRE,fsanna,-1,1873,1873,A00,0+1,Time forfeit,1. Nc3 { [%eval -0.07] } 1... e6 { [%eval 0.24...,A,"[-0.07, 0.24, -0.55, -0.22, -0.54, 0.0, -0.22,..."


In [28]:
chess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80523 entries, 0 to 80522
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   White        80523 non-null  object
 1   Black        80523 non-null  object
 2   Result       80523 non-null  int64 
 3   WhiteElo     80523 non-null  int64 
 4   BlackElo     80523 non-null  int64 
 5   ECO          80523 non-null  object
 6   TimeControl  80523 non-null  object
 7   Termination  80523 non-null  object
 8   AN           80523 non-null  object
 9   ECO_Family   80523 non-null  object
 10  evals        80523 non-null  object
dtypes: int64(3), object(8)
memory usage: 6.8+ MB


In [29]:
chess.to_csv("../data/raw/df_chess.csv", index=False)